In [195]:
import thermoextrap.xtrapy.xpan_beta as xpan_beta
import xarray as xr
import numpy as np
import thermoextrap.xtrapy.core as xtrapy_core
import thermoextrap.xtrapy.xstack as xstack

In [205]:
from importlib import reload
reload(xstack)

<module 'thermoextrap.xtrapy.xstack' from '/Users/wpk/Documents/python/projects/thermodynamic-extrapolation/thermoextrap/xtrapy/xstack.py'>

In [206]:
# synthetic data
# pretending data for rdf(pair, r)
# here, pair is for a-a, a-b, b-b
shape = (3, 2, 4)
dims = ['rec', 'pair', 'position']

coords = {'position': np.linspace(0, 2, shape[-1]), 'pair': ['a','b']}

xems = []
for beta in [0.1, 10.0]:
    x = xr.DataArray(np.random.rand(*shape), dims=dims, coords=coords)
    u = xr.DataArray(np.random.rand(shape[0]), dims=dims[0])
    data = xpan_beta.DataCentralMomentsVals.from_vals(x, u, order=3, central=True)
    xems.append(xpan_beta.factory_extrapmodel(beta, data))

s = xtrapy_core.StateCollection(xems)

In [199]:
# resample the data
sr = s.resample(nrep=5)

In [207]:
out = xstack.GPFlowData(sr, 
                        xdims=['beta','order'],
                        ydims=None, #will get from the rest
                       )

In [213]:
out.xarray

array([[ 0.1,  0. ],
       [ 0.1,  1. ],
       [ 0.1,  2. ],
       [ 0.1,  3. ],
       [10. ,  0. ],
       [10. ,  1. ],
       [10. ,  2. ],
       [10. ,  3. ]])

In [210]:
out.stacked

<xarray.DataArray (xstack: 8, ystack: 8, variable: 2)>
array([[[ 8.74444751e-01,  7.75353651e-03],
        [ 6.71454074e-01,  1.09328348e-02],
        [ 5.30869189e-01,  1.75565592e-02],
        [ 7.47437730e-01,  1.27650501e-03],
        [ 1.68843924e-01,  1.11174640e-02],
        [ 2.43907244e-01,  5.52396791e-03],
        [ 5.43083092e-01,  1.99308667e-02],
        [ 3.11633221e-01,  3.24061395e-03]],

       [[ 1.00304299e-02,  1.49042489e-04],
        [-2.72035994e-02,  1.57341854e-03],
        [-2.20790895e-02,  1.80703827e-03],
        [ 9.33810845e-03,  1.84203290e-04],
        [ 3.28621587e-02,  1.51334620e-03],
        [-1.13144495e-02,  1.21760098e-04],
        [-9.29374975e-03,  1.15542311e-03],
        [ 1.07345883e-02,  9.62857110e-05]],

       [[-5.19716664e-04,  7.32181104e-07],
        [-6.18379000e-03,  5.89052056e-05],
...
        [-2.67625231e-03,  4.77488428e-06],
        [ 3.58114491e-03,  8.54973259e-06]],

       [[-1.06669702e-03,  7.58561684e-07],
        [-9.09731074e-03,  5.51740418e-05],
        [ 6.56553689e-03,  2.87375164e-05],
        [ 4.83895862e-03,  1.56103470e-05],
        [-5.36800774e-04,  1.92103381e-07],
        [ 5.81724335e-03,  2.25602134e-05],
        [-4.29491004e-03,  1.22975015e-05],
        [-2.26996433e-03,  3.43515869e-06]],

       [[ 4.74335273e-03,  1.49995967e-05],
        [ 5.88063146e-03,  2.30545509e-05],
        [-6.16126120e-03,  2.53074264e-05],
        [-2.01887580e-03,  2.71723965e-06],
        [-2.17189614e-03,  3.14475524e-06],
        [ 1.02641032e-04,  7.02345434e-09],
        [ 3.62938277e-04,  8.78161284e-08],
        [-4.85654719e-04,  1.57240337e-07]]])
Coordinates:
  * variable  (variable) object 'mean' 'var'
  * xstack    (xstack) MultiIndex
  - beta      (xstack) float64 0.1 0.1 0.1 0.1 10.0 10.0 10.0 10.0
  - order     (xstack) int64 0 1 2 3 0 1 2 3
  * ystack    (ystack) MultiIndex
  - pair      (ystack) object 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b'
  - position  (ystack) float64 0.0 0.6667 1.333 2.0 0.0 0.6667 1.333 2.0

In [211]:
# loop over the ystacked variables if need be
for meta, g in out.stacked.groupby(out.yname):
    print(meta, g.shape)

('a', 0.0) (8, 2)
('a', 0.6666666666666666) (8, 2)
('a', 1.3333333333333333) (8, 2)
('a', 2.0) (8, 2)
('b', 0.0) (8, 2)
('b', 0.6666666666666666) (8, 2)
('b', 1.3333333333333333) (8, 2)
('b', 2.0) (8, 2)


In [ ]:
# I'll worry about conforming in the next iteration